## Задача

**Выявление инфраструктуры на спутниковых снимках с высоким разрешением.**

### Изучение данных и педобработка

In [ ]:
import cv2
import numpy as np
import pandas as pd
import cv2
from scipy import io
#import tensorflow_datasets as tfds
import tensorflow as tf
#from tensorflow import keras
import matplotlib.pyplot as plt
from PIL import Image
import PIL
import os
import re
from keras.models import load_model
import keras_segmentation
import types
from keras_segmentation.predict import predict
from keras_segmentation.models.unet import vgg_unet
from sklearn.metrics import classification_report

In [ ]:
def print_image_mask(image_path, mask_path):
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, 0)   
    res = cv2.bitwise_and(image, image, mask=mask)
    fig, ax = plt.subplots(1,2,figsize=(18,8))
    ax[0].imshow(image)
    ax[1].imshow(res)

In [ ]:
def print_3_pic(image_path, mask_path, path):
    image = cv2.imread(image_path)
    mask = cv2.imread(mask_path, 0)
    pic = cv2.imread(path)
    fig, ax = plt.subplots(1,3,figsize=(12,6))
    ax[0].imshow(image)
    res = cv2.bitwise_and(image, image, mask=mask)
    ax[1].imshow(res)
    ax[2].imshow(pic)

In [ ]:
def adjust_contrast_brightness(img, contrast:float=10.0, brightness:int=0):
    """
    Adjusts contrast and brightness of an uint8 image.
    contrast:   (0.0,  inf) with 1.0 leaving the contrast as is
    brightness: [-255, 255] with 0 leaving the brightness as is
    """
    brightness += int(round(255*(1-contrast)/2))
    '''
    low_green = np.array([25, 52, 72])
    high_green = np.array([102, 255, 255])
    mask = cv2.inRange(imgHSV, low_green, high_green)
    mask = 255-mask
    img = cv2.bitwise_and(img, img, mask=mask)
    print(img)
    '''
    
    return cv2.addWeighted(img, contrast, img, 0, brightness)

In [ ]:
def cut_pic (path, p_mask, splitter, extract, index):
    image = cv2.imread(path)
    mask = cv2.imread(p_mask, 0)
    h, w = image.shape[0:2]
    img_oi = []
    mask_oi = []
    if image.shape[0:2] == mask.shape[0:2]:
        jndex = 0
        h, w = image.shape[0:2]
        print("equal", r'h =', h, 'w =', w)
        (y, x) = (h // splitter, w // splitter)
        for i in range(splitter):
            for j in range(splitter):
                if (j < splitter - 1):
                    start_y = i*y
                    end_y = i*y+y
                    start_x = j*x
                    end_x = j*x+x
                else:
                    start_y = i*y+y
                    end_y = h
                    start_x = j*x+x
                    end_x = w   
                #print(start_y, ":", end_y, " / ",start_x, ":", end_x)
                i_tmp = image[start_y:end_y, start_x:end_x]
                m_tmp = mask[start_y:end_y, start_x:end_x]
                masked_image= cv2.bitwise_and(i_tmp, i_tmp, mask=m_tmp)
                # Считаем количество ненулевых пикселей в маске
                pixel_count = np.count_nonzero(masked_image)
                if (pixel_count > 40000):
                    img_oi.append(i_tmp)
                    mask_oi.append(m_tmp)
                    
                    image_path = extract + "/images/" + str(index) + '_' + str(jndex) + '.png'
                    #cv2.imwrite(image_path, i_tmp)
                    mask_path = extract + "/masks/" + str(index) + '_' + str(jndex) + '.png'
                    m_tmp = cv2.cvtColor(m_tmp, cv2.COLOR_GRAY2RGB)
                    #cv2.imwrite(mask_path, m_tmp)
                    bright_path = extract + "/bright/"  + str(index) + '_' + str(jndex) + '.png'
                    #cv2.imwrite(bright_path, adjust_contrast_brightness(i_tmp))
                    jndex +=1
                    if ((j % 2 == 0) & (i % 3 == 0)):
                        print_3_pic(image_path, mask_path, bright_path)
                    #print(0*3*((end_y - start_y)*(end_x - start_x)),pixel_count,  0.7 *((end_y - start_y)*(end_x - start_x)))
            
        #print(i*y+y, ":", h, " / ",j*x+x, ":", w)
    else:
        print("not equal")
    print(len(img_oi), len(mask_oi))
    return img_oi, mask_oi

In [ ]:
#path = "C:/Users/Maria Smirnova/Desktop/Проекты/satellite data/train/images"
path = "train/images"
index = 0
train_images =[]
train_masks = []
for address, dirs, files in os.walk(path):
    for f in files:
        image_path = address + '/' + f
        mask_path = re.sub('images', 'masks', address)
        mask_path += '/' + re.sub('image', 'mask', f)
        #print(image_path, mask_path)
        img, msk =cut_pic(image_path, mask_path, 20, "train/processing/", index)
        train_images += img
        train_masks += msk
        index +=1

### Пронозировнаие и оценка качества

In [ ]:
model_path = "train/processing/saved_models/30epoch_vgg_unet_segmentation.h5"
model = load_model(model_path)
model.predict_segmentation = types.MethodType(keras_segmentation.predict.predict, model)

In [ ]:
model_path = "train/processing/saved_models/1class_35epoch_vgg_unet_segmentation.hdf5"
n_classes = 2
model = vgg_unet(n_classes=n_classes)
model.load_weights(model_path)

In [ ]:
#path = "C:/Users/Maria Smirnova/Desktop/Проекты/satellite data/train/images"
path = "train/processing/val_images"
out_path = "train/processing/val_results"
for address, dirs, files in os.walk(path):
    for f in files:
        image_path = address + '/' + f
        mask_path = re.sub('images', 'masks', address)
        mask_path += '/' + re.sub('image', 'mask', f)
        print(image_path, mask_path)
        out = model.predict_segmentation(inp=image_path,
                                         out_fname=(out_path + '/' + f))
        #img, msk =cut_pic(image_path, mask_path, 20, "train/processing/", index)

In [ ]:
import tensorflow as tf
import keras.backend as K

def f1_loss(y_true, y_pred):

    tp = K.sum(K.cast(y_true*y_pred, 'float'), axis=0)
    tn = K.sum(K.cast((1-y_true)*(1-y_pred), 'float'), axis=0)
    fp = K.sum(K.cast((1-y_true)*y_pred, 'float'), axis=0)
    fn = K.sum(K.cast(y_true*(1-y_pred), 'float'), axis=0)

    p = tp / (tp + fp + K.epsilon())
    r = tp / (tp + fn + K.epsilon())

    f1 = 2*p*r / (p+r+K.epsilon())
    #f1 = tf.where(tf.is_nan(f1), tf.zeros_like(f1), f1)
    return 1 - K.mean(f1)

In [ ]:
path = "train/processing/val_images"
mask_path = "train/processing/val_masks"
out_path = "train/processing/val_results"
for address, dirs, files in os.walk(path):
    for f in files:
        y_true = np.array(cv2.imread(out_path+'/'+f))
        y_pred = np.array(cv2.imread(mask_path+'/'+f))
        print(tf.keras.backend.get_value(f1_loss(y_true, y_pred)))
        print_3_pic(path+'/'+f, mask_path+'/'+f, out_path+'/'+f)

### Пайплайн тестового датасета  

Расчитывался в ДВМ (DataSphere)

In [ ]:
path = "/home/jupyter/datasphere/project/test/test_image_00"
for i in range(8):
    print(i)
    out = model.predict_segmentation(
    inp=(path + str(i) + ".png"),
    out_fname="test_output_00" + str(i) + ".png")

### Обучение модели в DataSphere

In [ ]:
from tensorflow import keras
import numpy as np
import tensorflow as tf
import cv2
from scipy import io
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
from PIL import Image
import PIL
from keras_segmentation.models.unet import vgg_unet

In [ ]:
from keras_segmentation.models.unet import vgg_unet
from tensorflow.keras.metrics import Accuracy, Precision, Recall

img_url = "/home/jupyter/datasphere/project/images"
msk_url = "/home/jupyter/datasphere/project/masks"

epochs = 30

n_classes = 2
model = vgg_unet(n_classes=n_classes)
model.train(train_images = img_url, train_annotations = msk_url, 
            checkpoints_path = "/home/jupyter/datasphere/project", epochs=epochs)

model.save('/home/jupyter/datasphere/project/100epoch_vgg_unet_segmentation.h5')